<a href="https://colab.research.google.com/github/Mark-THU/load_forecast/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prediction by Transformer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sqlalchemy import create_engine
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"
device = torch.device(dev)

In [ ]:
# load data
# url = 'https://raw.githubusercontent.com/Mark-THU/load_forecast/main/integrate_0101510000.csv'
url = 'https://raw.githubusercontent.com/Mark-THU/load_forecast/main/dataset.csv'
data = pd.read_csv(url, sep='\t', index_col='time')
data = data[['tem', 'tembody', 'month_of_year', 'is_holiday', 'day_of_week', 'load']]

In [ ]:
# 归一化
def normalization(data):
    """
    data: original data with load
    return: normalized data, scaler of load
    """
    normalized_data = MinMaxScaler().fit_transform(data)
    scaler_y = MinMaxScaler()
    scaler_y.fit_transform(data[['load']])
    return normalized_data, scaler_y

In [ ]:
# build supervised data
def Series_To_Supervise(data, seq_len, target_len, y_col_index):
    """
    convert series data to supervised data
    :param data: original data
    :param seq_len: length of sequence
    :y_col_index: index of column which acts as output
    :return: return two ndarrays-- input and output in format suitable to feed to LSTM
    """
#     pdb.set_trace()
    dim_0 = data.shape[0] - seq_len
    dim_1 = data.shape[1]
    x = np.zeros((dim_0, seq_len, dim_1))
    y = np.zeros((dim_0, target_len))
    for i in range(dim_0):
        x[i] = data[i: i+seq_len]
        y[i] = data[i+seq_len+1-target_len:i+seq_len+1, y_col_index]
    print("shape of x: {}, shape of y: {}".format(x.shape, y.shape))
    return x, y

In [ ]:
# 5-fold cross-validation
def split_dataset(X, Y, n_split=5):
    """
    X: original feature, size * 72 * features
    Y: labels, size * 1
    return: list of train_x, test_x, train_y, test_y
    """
    kf = KFold(n_splits=n_split, shuffle=True, random_state=1)
    train_x_list = list()
    valid_x_list = list()
    test_x_list = list()
    train_y_list = list()
    valid_y_list = list()
    test_y_list = list()
    for train_index, test_index in kf.split(X):
        train_x_list.append(X[train_index])
        train_y_list.append(Y[train_index])
        test_x = X[test_index]
        test_y = Y[test_index]
        valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=1)
        valid_x_list.append(valid_x)
        valid_y_list.append(valid_y)
        test_x_list.append(test_x)
        test_y_list.append(test_y)
    return train_x_list, valid_x_list, test_x_list, train_y_list, valid_y_list, test_y_list

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
# define encoder
class Encoder(nn.Module):
    def __init__(self, input_size=13, d_model=64, nhead=8, n_layers=4, dim_feedward=256):
        super(Encoder, self).__init__()
        
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model=d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedward)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)
    def forward(self, x):
        embedded = self.embedding(x)
        encoder_input = self.pos_encoder(embedded)
        output = self.transformer_encoder(encoder_input)
        return output

In [ ]:
def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence. The masked positions are filled with float(‘-inf’). Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.to(device)

In [ ]:
# define decoder
class Decoder(nn.Module):
    def __init__(self, input_size=13, d_model=64, nhead=8, n_layers=4, dim_feedward=256):
        super(Decoder, self).__init__()
        
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedward)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=n_layers)
        self.out = nn.Linear(d_model, 1)
    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        embedded = self.embedding(tgt)
        decoder_input = self.pos_encoder(embedded)
        output = self.transformer_decoder(decoder_input, memory, tgt_mask, memory_mask)
        output = self.out(output)[-1,:]
        return output        

In [ ]:
# train the model 
def train_model(train_x, train_y, valid_x, valid_y, input_size=13, d_model=32, nhead=8, n_layers=4, dim_feedward=128,
                seq_len=48, target_len=24, number_epochs=50, batch_size=512, teacher_forcing_ratio=0.5,
                lr=0.01, training_prediction='teacher_forcing', dynamic_tf=False):
    encoder = Encoder(input_size, d_model, nhead, n_layers, dim_feedward)
    decoder = Decoder(input_size, d_model, nhead, n_layers, dim_feedward)
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    train_dataset = TensorDataset(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
    valid_dataset = TensorDataset(torch.FloatTensor(valid_x), torch.FloatTensor(valid_y))
    criterion = nn.MSELoss()
    encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    train_losses = list()
    valid_loss_min = np.Inf
    num_without_imp = 0
    y_index = train_x.shape[2] - 1
    
    # train
    for epoch in range(1, number_epochs + 1):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.transpose(0, 1)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = torch.zeros(batch_size, target_len).to(device)
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            # encoder
            input_tensor = inputs[0:seq_len, :, :]
            target_tensor = inputs[seq_len:, :, :]
            memory = encoder(input_tensor)
            # decoder
            tgt = inputs[seq_len-1, :, :].unsqueeze(0)
            
            if training_prediction == 'recursive':
                # predict recursively
                for t in range(target_len):
                    tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                    output = decoder(tgt, memory, tgt_mask=tgt_mask)
                    outputs[:, t] = output.squeeze()
                    added_input = torch.cat((target_tensor[t, :, :-1].unsqueeze(0),
                                             output.detach().unsqueeze(0)), 2)
                    tgt = torch.cat((tgt, added_input), 0)
            
            if training_prediction == 'teacher_forcing':
                # use teacher forcing
                if random.random() < teacher_forcing_ratio:
                    for t in range(target_len):
                        tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                        output = decoder(tgt, memory, tgt_mask=tgt_mask)
                        outputs[:, t] = output.squeeze()
                        added_input = target_tensor[t, :, :].unsqueeze(0)
                        tgt = torch.cat((tgt, added_input), 0)
                # predict recurisively
                else:
                    for t in range(target_len):
                        tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                        output = decoder(tgt, memory, tgt_mask=tgt_mask)
                        outputs[:, t] = output.squeeze()
                        added_input = torch.cat((target_tensor[t, :, :-1].unsqueeze(0),
                                                 output.detach().unsqueeze(0)), 2)
                        tgt = torch.cat((tgt, added_input), 0)
            
            if training_prediction == 'mixed_teacher_forcing':
                # predict using mixed teacher forcing
                for t in range(target_len):
                    tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                    output = decoder(tgt, memory, tgt_mask=tgt_mask)
                    outputs[:, t] = output.squeeze()
                    # predict with teacher forcing
                    if random.random() < teacher_forcing_ratio:
                        added_input = target_tensor[t, :, :].unsqueeze(0)
                        tgt = torch.cat((tgt, added_input), 0)
                            
                    # predict recursively 
                    else:
                        added_input = torch.cat((target_tensor[t, :, :-1].unsqueeze(0),
                                                 output.detach().unsqueeze(0)), 2)
                        tgt = torch.cat((tgt, added_input), 0)
            loss = criterion(outputs, labels)
            train_losses.append(loss.item)
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            
            # eval
            if i % 5 == 0:
                if num_without_imp > 10:
                    return encoder, decoder
                num_without_imp = num_without_imp + 1
                valid_losses = list()
                encoder.eval()
                decoder.eval()
                for inp, lab in valid_loader:
                    inp = inp.transpose(0, 1)
                    inp = inp.to(device)
                    lab = lab.to(device)
                    out = torch.zeros(batch_size, target_len).to(device)
                    # encoder
                    input_tensor = inp[0:seq_len, :, :]
                    target_tensor = inp[seq_len:, :, 0:-1]
                    memory = encoder(input_tensor)
                    # decoder
                    tgt = inp[seq_len-1, :, :].unsqueeze(0)
                    # predict recuisively
                    for t in range(target_len):
                        tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                        output = decoder(tgt, memory, tgt_mask=tgt_mask)
                        out[:, t] = output.squeeze()
                        added_input = torch.cat((target_tensor[t, :, :].unsqueeze(0),
                                                 output.detach().unsqueeze(0)), 2)
                        tgt = torch.cat((tgt, added_input), 0)
                    valid_loss = criterion(out, lab)
                    valid_losses.append(valid_loss.item())
                encoder.train()
                decoder.train()
                print("Epoch: {}/{}...".format(epoch, number_epochs),
                     "Step: {}/{}...".format(i+1, len(train_dataset)//batch_size),
                     "Loss: {}...".format(loss.item()),
                     "Valid Loss: {}...".format(np.mean(valid_losses)))
                if np.mean(valid_losses) < valid_loss_min:
                    num_without_imp = 0
                    torch.save(encoder.state_dict(), "encoder_state_dict.pt")
                    torch.save(decoder.state_dict(), "decoder_state_dict.pt")
                    print("Valid loss decreased ({:.6f} --> {:.6f}).  Saving model ...".format(valid_loss_min, np.mean(valid_losses)))
                    valid_loss_min = np.mean(valid_losses)
    return encoder, decoder

In [ ]:
# test the model
def test_model(encoder, decoder, test_x, test_y, batch_size, seq_len, target_len, scaler):
    test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    encoder.load_state_dict(torch.load('encoder_state_dict.pt'))
    decoder.load_state_dict(torch.load('decoder_state_dict.pt'))
    y_pred = list()
    y_true = list()
    y_index = test_x.shape[2] - 1
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.transpose(0, 1)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = torch.zeros(batch_size, target_len).to(device)
            # encoder
            input_tensor = inputs[0:seq_len, :, :]
            target_tensor = inputs[seq_len:, :, :]
            memory = encoder(input_tensor)
            # decoder
            tgt = inputs[seq_len-1, :, :].unsqueeze(0)
            # predict recursively
            for t in range(target_len):
                tgt_mask = generate_square_subsequent_mask(tgt.shape[0])
                output = decoder(tgt, memory, tgt_mask=tgt_mask)
                outputs[:, t] = output.squeeze()
                added_input = torch.cat((target_tensor[t, :, :-1].unsqueeze(0),
                                         output.detach().unsqueeze(0)), 2)
                tgt = torch.cat((tgt, added_input), 0)
            y_pred = y_pred + outputs.view(-1).cpu().numpy().tolist()
            y_true = y_true + labels.view(-1).cpu().numpy().tolist()
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    load_true = scaler.inverse_transform(np.expand_dims(y_true, axis=1))
    load_pred = scaler.inverse_transform(np.expand_dims(y_pred, axis=1))
    MAPE = np.mean(np.abs(load_true-load_pred)/load_true)
    return MAPE

In [ ]:
# model configs
def Model_Configs():
    batch_sizes = [512]
    n_layers = [1]
    lrs = [0.01]
    hidden_dims = [70]
    configs = list()
    for i in batch_sizes:
        for j in n_layers:
            for k in lrs:
                for l in hidden_dims:
                    configs.append([i, j, k, l])
    return configs

In [ ]:
def main(n_split=5, seq_len=48, target_len=24):
    normalized_data, scaler_y = normalization(data)
    y_index = normalized_data.shape[1] - 1
    num_features = normalized_data.shape[1]
    x, y = Series_To_Supervise(normalized_data, seq_len=seq_len + target_len, target_len=target_len, y_col_index=y_index)
    train_x_list, valid_x_list, test_x_list, train_y_list, valid_y_list, test_y_list = split_dataset(x, y, n_split=n_split)    
    print("model configs set.")
    configs = Model_Configs()
    MAPE_list = list()
    for config in configs:
        batch_size = config[0]
        n_layer = config[1]
        lr = config[2]
        hidden_dim = config[3]
        print("Config: batch_size--{}, n_layer--{}, lr--{}, hidden_dims--{}".format(batch_size, n_layer, lr, hidden_dim))
        tmp_list = list()
        tmp_list_24 = list()
        for i in range(n_split):
            encoder, decoder = train_model(train_x_list[i], train_y_list[i], valid_x_list[i], valid_y_list[i],
                                           input_size=num_features, batch_size=batch_size, lr=lr, seq_len=seq_len, 
                                           target_len=target_len, training_prediction='teacher_forcing')
            MAPE = test_model(encoder, decoder, test_x_list[i], test_y_list[i], batch_size=batch_size, seq_len=seq_len,
                              target_len=target_len, scaler=scaler_y)
            if MAPE < 0.1:
                break
            tmp_list.append(MAPE)
        MAPE_list.append(tmp_list)
    return MAPE_list

In [ ]:
MAPE_list = main()

shape of x: (22745, 72, 6), shape of y: (22745, 24)
model configs set.
Config: batch_size--512, n_layer--1, lr--0.01, hidden_dims--70
Epoch: 1/50... Step: 1/35... Loss: 0.36865609884262085... Valid Loss: 5.518253564834595...
Valid loss decreased (inf --> 5.518254).  Saving model ...
Epoch: 1/50... Step: 6/35... Loss: 0.06205814331769943... Valid Loss: 0.05728647764772177...
Valid loss decreased (5.518254 --> 0.057286).  Saving model ...
Epoch: 1/50... Step: 11/35... Loss: 0.030360614880919456... Valid Loss: 0.02581830369308591...
Valid loss decreased (0.057286 --> 0.025818).  Saving model ...
Epoch: 1/50... Step: 16/35... Loss: 0.04631965979933739... Valid Loss: 0.03129823878407478...
Epoch: 1/50... Step: 21/35... Loss: 0.036569248884916306... Valid Loss: 0.035850342363119125...
Epoch: 1/50... Step: 26/35... Loss: 0.028457563370466232... Valid Loss: 0.020985573064535856...
Valid loss decreased (0.025818 --> 0.020986).  Saving model ...
Epoch: 1/50... Step: 31/35... Loss: 0.027755616232

In [41]:
MAPE_list

[[0.20634211810239275,
  0.19461447828968573,
  0.19300887313337248,
  0.19481059123145575,
  0.20026291033115215]]

In [ ]:
configs = Model_Configs()
best_config = configs[np.argmin(MAPE_list)]
for i in range(len(configs)):
    print("config: {}, MAPE: {:.6f}".format(configs[i], MAPE_list[i]))
print("best config: {}, MAPE: {:.6f}".format(best_config, min(MAPE_list)))

IndexError: ignored